In [ ]:
# In this example, we will load monthly energy data for one buiding for 2022 and 2023 and hourly weather data for
# a nearby weather station in Rogers, AR. First, we will calculate the cooling and heating degree days using a 
# base temperature of 65 degrees. Then we will weather normalize our energy data using the HDD and CDD data and
# using 2022 as a baseline year. Then we'll predict energy usage for 2023 and compare our predicted usage to the
# actual usage for 2023 to see if there were potential energy savings in 2023.

import pandas as pd

# Load the CSV data
df = pd.read_csv('Reference Materials/Rogers hourly weather data.csv')

In [ ]:
# Define the base temperature as 65, which we will use to calculate both HDD and CDD in this example
base_temp = 65

# Convert the datetime column to datetime objects and set it as the index
df['datetime_beginning'] = pd.to_datetime(df['datetime_beginning'])
df.set_index('datetime_beginning', inplace=True)

# Calculate the daily average temperature from the hourly temperature values in our data set
daily_avg_temp = df['temp_f'].resample('D').mean()

# Initialize HDD and CDD Series with zeros
hdd = pd.Series(0, index=daily_avg_temp.index)
cdd = pd.Series(0, index=daily_avg_temp.index)

# Calculate HDD and CDD using the 65 degree base temperature
for day in daily_avg_temp.index:
    avg_temp = daily_avg_temp[day]
    if avg_temp < base_temp:
        hdd[day] = base_temp - avg_temp
    elif avg_temp > base_temp:
        cdd[day] = avg_temp - base_temp

In [ ]:
# Create a new dataframe to align the calculated daily average temp, hdd, and cdd values by date
result_df = pd.DataFrame({
    'date': daily_avg_temp.index.date,
    'daily_avg_temp': daily_avg_temp.values,
    'hdd': hdd.values,
    'cdd': cdd.values
})

# Set the date as the index
result_df.set_index('date', inplace=True)

result_df.index = pd.to_datetime(result_df.index)

result_df

In [ ]:
#Let's take a look at our calculated average temperature, HDD, and CDD data for some number of days to see the relationship

import matplotlib.pyplot as plt
import seaborn as sns

# Set the number of days to display in the plot
number_of_days = 200

first_days = result_df.iloc[:number_of_days]

# Set the chart style
sns.set(style="whitegrid")

# Create a figure and a set of subplots
fig, ax1 = plt.subplots(figsize=(15, 7))

# Plot the HDD and CDD as bars on the primary y-axis
ax1.bar(first_days.index, first_days['hdd'], width=0.4, color='red', align='center', label='HDD (°F)', alpha=0.6)
ax1.bar(first_days.index, first_days['cdd'], width=0.4, color='blue', align='edge', label='CDD (°F)', alpha=0.6)

# Set the primary y-axis label
ax1.set_ylabel('Degree Days (°F)')
ax1.set_xlabel('Date')

# Remove horizontal gridlines for the primary axis
ax1.yaxis.grid(False)

# Create a secondary y-axis for the average daily temperature
ax2 = ax1.twinx()

# Plot the average daily temperature as a line on the secondary y-axis
ax2.plot(first_days.index, first_days['daily_avg_temp'], color='tab:blue', label='Avg Daily Temp (°F)')

# Set the secondary y-axis label
ax2.set_ylabel('Avg Daily Temp (°F)')

# Remove horizontal gridlines for the secondary axis
ax2.yaxis.grid(False)

# Set the title
ax1.set_title(f'Average Daily Temperature, HDD, and CDD for the First {number_of_days} Days')

# legend and ticks
h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
ax1.legend(h1+h2, l1+l2, loc='upper left')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Now let's load our building energy data that we want to weather normalize
file_path = 'Reference Materials/Building energy data 2022-2023.xlsx'

# Load the energy data into a dataframe
energy_data = pd.read_excel(file_path)

In [ ]:
# Convert the Month column in the energy data to datetime
energy_data['Month'] = pd.to_datetime(energy_data['Month'])

# Make sure the Month column in result_df is also datetime in the same format
result_df['Month'] = result_df.index.to_period('M').to_timestamp()

# Calculate the average monthly temperature in result_df so we can align it with monthly energy data
monthly_avg_temp = result_df.groupby('Month')['daily_avg_temp'].mean().reset_index()

# group result_df by Month and sum the HDD and CDD, and get the average temperature
monthly_hdd_cdd_temp = result_df.groupby('Month').agg({'hdd': 'sum', 'cdd': 'sum', 'daily_avg_temp': 'mean'}).reset_index()

# Merge the energy data with the monthly HDD and CDD data based on the Month column
combined_data = pd.merge(energy_data, monthly_hdd_cdd_temp, on='Month', how='left')

# Rename the 'daily_avg_temp' column to 'monthly_avg_temp'
combined_data.rename(columns={'daily_avg_temp': 'monthly_avg_temp'}, inplace=True)


In [ ]:
# Saving the data again so I can examine it in excel or use it as a future starting point if I want to
excel_filename = 'combined_data.xlsx'

combined_data.to_excel(excel_filename)

combined_data

In [ ]:
# Now we will building a linear regression model of the monthly energy data regressed on HDD and CDD
# We will do this only for the year 2022, as that will be our baseline year if we want to estimate savings in 2023

import statsmodels.api as sm

# Filter the dataset for the baseline year (2022)
baseline_data = combined_data[combined_data['Month'].dt.year == 2022]

# Define the dependent variable for the baseline year
y_baseline = baseline_data['Energy_kBTU']

# Define the independent variables for the baseline year
X_baseline = baseline_data[['hdd', 'cdd']]

# Add a constant to the independent variable set
X_baseline = sm.add_constant(X_baseline)

# Fit the regression model on the baseline year data
model_baseline = sm.OLS(y_baseline, X_baseline).fit()

# Print the summary of the regression for the baseline year
print(model_baseline.summary())


In [ ]:
# # Now we will use our model to calculate predicted usage for 2023 based on the HDD and CDD values for that year
# Our predicted usage is our weather-normalized usage, so it should allow us to see if we saved energy in 2023 or used
# more than expected.

# Filter the dataset for the prediction year (2023)
prediction_data = combined_data[combined_data['Month'].dt.year == 2023]

# Prepare the independent variables for the prediction year
X_prediction = prediction_data[['hdd', 'cdd']]
X_prediction = sm.add_constant(X_prediction)  # Add a constant to the variable set

# Use the model from the baseline year to make predictions for the prediction year
prediction_data['predicted_Energy_kBTU'] = model_baseline.predict(X_prediction)

# Ensure 'Month' is the index for both dataframes
combined_data.set_index('Month', inplace=True)
prediction_data.set_index('Month', inplace=True)

# Add the predictions to the combined_data dataframe
combined_data.loc[prediction_data.index, 'predicted_Energy_kBTU'] = prediction_data['predicted_Energy_kBTU']

combined_data

In [ ]:
# Now we can calculate predicted - actual usage for each month in 2023

# Reset the index of combined_data to make 'Month' a column again
combined_data.reset_index(inplace=True)

# Filter down to just 2023 data
data_2023 = combined_data[combined_data['Month'].dt.year == 2023]

# Calculate the difference between predicted and actual values energy
data_2023['difference_kBTU'] = data_2023['predicted_Energy_kBTU'] - data_2023['Energy_kBTU']

# Calculate sum total of the difference for 2023
total_difference_kBTU = data_2023['difference_kBTU'].sum()

print(data_2023[['Month', 'predicted_Energy_kBTU', 'Energy_kBTU', 'difference_kBTU']])
print(f"Total difference (Predicted - Actual) for 2023: {total_difference_kBTU} kBTU")

In [ ]:
# Let's plot our actual usage for 2022 and 2023 and our predicted usage for just 2023 since 2022 was our baseline year
# which means we didn't have any predicted, or weather normalized, usage for that year

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Ensure that 'Month' is a datetime column and sort the dataframe by 'Month'
combined_data['Month'] = pd.to_datetime(combined_data['Month'])
combined_data.sort_values('Month', inplace=True)

# Plot actual Energy_kBTU for 2022 and 2023
plt.figure(figsize=(12, 6))
plt.plot(combined_data['Month'], combined_data['Energy_kBTU'], label='Actual Energy_kBTU', color='blue', marker='o')

# Plot predicted Energy_kBTU for 2023
plt.plot(combined_data['Month'], combined_data['predicted_Energy_kBTU'], label='Predicted Energy_kBTU', color='red', linestyle='--', marker='x')

# Format the plot
plt.title('Actual vs Predicted Energy Consumption (kBTU)')
plt.xlabel('Month')
plt.ylabel('Energy (kBTU)')
plt.legend()
plt.grid(True)

# Update the x-axis labels to show month names
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))
plt.gcf().autofmt_xdate()

plt.show()
